In [6]:
# импортируем библиотеки numpy и pandas
import numpy as np
import pandas as pd
# импортируем классы BaseEstimator и TransformerMixin,
# позволяющие написать собственные классы
from sklearn.base import BaseEstimator, TransformerMixin
# импортируем классы, необходимые для корректной работы
# пользовательских классов RareGrouper и RareGrouper2
from pandas import Series, DataFrame
# импортируем класс SimpleImputer, позволяющий
# выполнить импутацию пропусков
from sklearn.impute import SimpleImputer


Создание класса импутирования средним по группам

In [7]:
class MeanGroupImputer(BaseEstimator, TransformerMixin):
    """
    Параметры:

    copy: bool, по-умолчанию True
         Если True, то возвращает копию.
    """
    def __init__(self, copy=True):
        self.copy = copy

    def fit(self, X, y=None, group_col=None):
        self._encoder_dict = {}

        self._uniq = X[group_col].unique().tolist()

        for col in X.columns:
            if col != group_col:
                gr = {}
                for group in self._uniq:
                    gr[group] = X.loc[X[group_col]==group, col].mean()
                self._encoder_dict[col] = gr

        return  self

    def transform(self, X, group_col=None):
        if self.copy:
            X = X.copy()

        for col in X.columns:
            if col != group_col:
                for group in self._uniq:
                    X.loc[X[group_col]==group, col] = np.where(X.loc[X[group_col]==group, col].isnull(), self._encoder_dict[col][group], X.loc[X[group_col]==group, col] )

        return X

Создание фиктивного датасета

In [8]:
data = pd.DataFrame({'Value': [2, np.NaN, 4, 3, np.NaN, 5, 7, np.NaN, 9],
                     'Category': ['cat1', 'cat1', 'cat1', 'cat2', 'cat2', 'cat2', 'cat3', 'cat3', 'cat3']})
data

,Value,Category
0,2.0,cat1
1,NaN,cat1
2,4.0,cat1
3,3.0,cat2
4,NaN,cat2
5,5.0,cat2
6,7.0,cat3
7,NaN,cat3
8,9.0,cat3


In [9]:
mgi = MeanGroupImputer()
mgi.fit(data, group_col='Category')
mgi._encoder_dict

{'Value': {'cat1': 3.0, 'cat2': 4.0, 'cat3': 8.0}}

In [10]:
data_trans = mgi.transform(data, group_col='Category')
data_trans

,Value,Category
0,2.0,cat1
1,3.0,cat1
2,4.0,cat1
3,3.0,cat2
4,4.0,cat2
5,5.0,cat2
6,7.0,cat3
7,8.0,cat3
8,9.0,cat3
